## Bag of Words(BoW)


***
**- 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법**

- **백오브워즈** 가정에서는 어떤 단어가 많이 쓰였는지 정보를 중시한다.<br>
저자의 의도는 단어 사용 여부나 그 빈도에서 드러난다고 보기 때문이다. 단어의 순서 정보는 무시한다.


1) 각 단어에 고유한 정수 인덱스를 부여한다.

2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.



In [9]:
from konlpy.tag import Okt
import re

okt = Okt()
token = re.sub("[.]","","지금은 6시 3분 오늘은 많은 어려움을 겪었습니다.")
token = okt.morphs(token)


word2index = {}
bow = []

for voca in token :
    if voca not in word2index.keys() :
        word2index[voca] = len(word2index)
        
        bow.insert(len(word2index)-1,1)

    else :
        index = word2index.get(voca)
        # 재등장하는 단어의 인덱스를 받아옵니다.
        bow[index]=bow[index]+1
# 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다. (단어의 개수를 세는 것입니다.)  
print(word2index)  
        
        

{'지금': 0, '은': 1, '6시': 2, '3분': 3, '오늘': 4, '많은': 5, '어려움': 6, '을': 7, '겪었습니다': 8}


**word2index**를 실행했더니 각 단어에 대해서 인덱스를 부여한 결과가 출력되었다.

In [10]:
bow

[1, 2, 1, 1, 1, 1, 1, 1, 1]

'은'이 2번 언급되었기 때문에 인덱스 2의 값이 2임을 알 수 있다.<br>

한국어에서 불용어에 해당되는 조사들을 제거하여 더 정제된 BoW를 만들 수도 있다.

BoW에서 중요한 것은 단어의 등장 빈도다. 따라서 단어의 순서, 즉, 인덱스의 순서는 전혀 상관이 없다. 따라서 인덱스 할당을 임의로 바꿔도 개념은 같다.

## 2. CounterVectorizer 클래스로 BoW 만들기

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


CountVectorizer는 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식한다.


 정제(Cleaning) 챕터에서 언급했듯이, 영어에서는 길이가 짧은 문자를 제거하는 것 또한 전처리 작업으로 고려되기도 한다.

## 3.문서 단어 행렬(Document-Term Matrix, DTM)

*백오브워즈 임베딩에는 저자가 생각한 주제가 문서에서의 단어 사용에 녹아 있다*는 가정이 깔려있다.
- 다시 말해 주제가 비슷한 문서라면 단어 빈도 또는 단어 등장 역시 비슷할 것이고, 백오브워즈 임베딩 역시 유사할 것이라고 보는 것이다.

- 빈도를 그대로 백오브워즈로 쓴다면 많이 쓰인 단어가 주제와 더 강한 관련을 맺고 있을 것이라는 전제 역시 깔려있다.


**문서 단어 행렬(DTM)이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말한다.**<br><br>


각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어라고 생각하면 좋다.




## 4. 문서 단어 행렬(Document-Term Matrix, DTM)의 한계


### 1) 희소 표현(Sparse representation)

단어 집합의 크기가 벡터의 차원이 되고 대부분의 값이 0이 된다는 특징을 가진 원-핫 벡터와 마찬가지로 DTM도 공간적 낭비와 계산 리소스를 증가시킬 수 있다.

원-핫 벡터나 DTM과 같은 대부분의 값이 0인 표현을 **희소 벡터(sparse vector) 또는 희소 행렬(sparse matrix)**라고 부르는데, 희소 벡터는 많은 양의 저장 공간과 계산을 위한 리소스를 필요로 한다.

이러한 이유로 전처리를 통해

단어 집합의 크기를 줄이는 일은 BoW 표현을 사용하는 모델에서 중요하다.

구두점, 빈도수가 낮은 단어, 불용어를 제거하고, 어간이나 표제어 추출을 통해 단어를 정규화하여 단어 집합의 크기를 줄이도록 해야한다.

### 2) 단순 빈도 수 기반 접근

각 문서에는 중요한 단어들과 불필요한 단어들이 혼재되어 있다. 그렇다보니 빈도를 기반으로 표기하는 방법은 때로는 한계를 가지기도 한다.

불용어는 자연어 처리에 있어 의미를 거의 갖지 못하는 단어이기 때문에 DTM에 불용어와 중요한 단어에 대해서 가중치를 줄 수 있는 방법을 찾아봐야한다.

그럴때 사용하는게 **TF-IDF**이다.
